# Stellar Mass by Region

In [73]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

In [74]:
from auriga.snapshot import Snapshot
from auriga.images import figure_setup, set_axs_configuration
from auriga.settings import Settings
from auriga.parser import parse

In [75]:
figure_setup()
settings = Settings()

In [76]:
DISC_STD_CIRC = settings.disc_std_circ
DISC_MIN_CIRC = settings.disc_min_circ
COLD_DISC_DELTA_CIRC = settings.cold_disc_delta_circ
BULGE_MAX_SPECIFIC_ENERGY = -0.6
SUFFIX = "_02"

In [77]:
def read_data(simulation: str) -> pd.DataFrame:
    """
    Return a DataFrame with data of interest.
    """
    settings = Settings()

    s = Snapshot(simulation=simulation, loadonlytype=[0, 1, 2, 3, 4, 5])
    s.tag_particles_by_region(
        disc_std_circ=DISC_STD_CIRC,
        disc_min_circ=DISC_MIN_CIRC,
        cold_disc_delta_circ=COLD_DISC_DELTA_CIRC,
        bulge_max_specific_energy=BULGE_MAX_SPECIFIC_ENERGY)

    is_real_star = (s.type == 4) & (s.stellar_formation_time > 0)
    is_main_obj = (s.halo == s.halo_idx) & (s.subhalo == s.subhalo_idx)

    df = pd.DataFrame()
    df["RegionTag"] = s.region_tag[is_real_star & is_main_obj]
    df["Mass_Msun"] = s.mass[is_real_star & is_main_obj]

    return df

In [78]:
def get_statistics(df: pd.DataFrame) -> tuple:
    total_solar_masses = df["Mass_Msun"].sum()
    components_solar_masses = np.zeros(4)
    for i in range(len(settings.components)):
        components_solar_masses[i] = df["Mass_Msun"][
            df["RegionTag"] == i].sum()
    return components_solar_masses / total_solar_masses

In [79]:
def get_min_and_max_values(sample: list) -> tuple:
    min_vals = np.ones(4, dtype=np.float64)
    max_vals = np.zeros(4, dtype=np.float64)
    for simulation in sample:
        data = get_statistics(read_data(simulation))
        max_vals[data > max_vals] = data[data > max_vals]
        min_vals[data < min_vals] = data[data < min_vals]
    return min_vals, max_vals

### Figures

In [80]:
# SIMULATION = "au6_or_l4_s127"
SIMULATION = None

In [81]:
SAMPLE = [f"au{i}_or_l4_s127" for i in settings.groups["Included"]]
# SAMPLE = None

In [82]:
if SIMULATION:
    settings = Settings()
    min_vals, max_vals = get_min_and_max_values(
        [f"au{i}_or_l4_s127" for i in settings.groups["Included"]])

    fig, ax = plt.subplots(figsize=(2.5, 2.5))

    ax.tick_params(which='both', direction="in")

    ax.set_xlim(-0.5, 3.5)
    ax.set_xticks([0, 1, 2, 3])
    ax.set_xticklabels([])

    ax.set_ylim(0, 1)
    ax.set_yticks([0, 0.2, 0.4, 0.6, 0.8, 1])
    ax.set_ylabel(r"$f_\star$")

    label = f"Au{parse(SIMULATION)[0]}"
    data = get_statistics(read_data(SIMULATION))

    for j in range(len(data)):
        yerr = np.abs(
            (np.array([min_vals[j], max_vals[j]]) - data[j]).reshape(2, 1))
        ax.bar(x=j, height=data[j], yerr=yerr,
               color=list(settings.component_colors.values())[j],
               width=0.5, linewidth=0)
        ax.text(j, data[j] + yerr[1] + 0.025,
                s=r"$\textbf{" + str(int(np.round(100 * data[j], 0))) \
                    + "\%" + "}$",
                c=list(settings.component_colors.values())[j],
                ha="center", va="bottom", size=8.0)
        ax.text(j, -0.05, size=8.0,
                s=r"$\textbf{" + settings.components[j] + "}$",
                c=list(settings.component_colors.values())[j],
                ha="center", va="top")

    ax.text(
        x=0.05, y=0.95, size=8.0,
        s=r"$\texttt{" + label + "}$",
        ha="left", va="top", transform=ax.transAxes)

    fig.savefig(
        f"../images/galaxy_decomposition/"
        f"stellar_mass_distribution_{SIMULATION}{SUFFIX}.pdf")
    plt.close(fig)

In [83]:
if SAMPLE:
    fig, axs = plt.subplots(figsize=(7, 8), nrows=6, ncols=4,
                            sharey=True, sharex=True,
                            gridspec_kw={"hspace": 0.0, "wspace": 0.0})

    for ax in axs.flat:
        ax.tick_params(which='both', direction="in")
        if ax == axs[-1, -1]: ax.axis("off")
        ax.set_xlim(-0.5, 3.5)
        ax.set_xticks([0, 1, 2, 3])
        ax.set_xticklabels([])
        ax.set_ylim(0, 1)
        ax.set_yticks([0, 0.2, 0.4, 0.6, 0.8])
        ax.set_axisbelow(True)
        if ax.get_subplotspec().is_last_row() or ax == axs[-2, -1]:
            ax.set_xlabel("")
            ax.tick_params(labelbottom=True)
        if ax.get_subplotspec().is_first_col():
            ax.set_ylabel(r"$f_\star$")

    for i, simulation in enumerate(SAMPLE):
        label = f"Au{parse(simulation)[0]}"
        data = get_statistics(read_data(simulation))
        ax = axs.flatten()[i]
        ax.text(
            x=0.05, y=0.95, size=6.0,
            s=r"$\texttt{" + label + "}$",
            ha="left", va="top", transform=ax.transAxes)

        for j in range(len(data)):
            ax.bar(x=j, height=data[j],
                   color=list(settings.component_colors.values())[j],
                   width=0.5, linewidth=0)
            ax.text(j, data[j] + 0.025,
                s=r"$\textbf{" + str(int(np.round(100 * data[j], 0))) \
                    + "\%" + "}$",
                c=list(settings.component_colors.values())[j],
                ha="center", va="bottom", size=5.0)
            
        if ax.get_subplotspec().is_last_row() or ax == axs[-2, -1]:
            ax.text(j, -0.05, size=6.0,
                    s=r"$\textbf{" + settings.components[j] + "}$",
                    c=list(settings.component_colors.values())[j],
                    ha="center", va="top")

        fig.savefig(
            f"../images/galaxy_decomposition/"
            f"stellar_mass_distribution_included{SUFFIX}.pdf")
    plt.close(fig)